## Filters

过滤器是资产和时刻的函数，它是一个布尔值：

![过滤器函数](./filter_fun.png)

在管道中，`Filters`用于缩小包含在计算中或流水线最终输出中的证券集合。有两种常用的方法来创建过滤器：`Factor/Classifier`方法。

## Comparison Operators

因子和分类器的比较运算产生过滤器。由于我们还没有看过`Classifiers`，所以让我们坚持使用因子的例子。以下示例会生成一个筛选器，只要最新收盘价格高于20，就会返回True。

这个例子产生一个过滤器，当10天平均值低于30天平均值时返回True。

请记住，过滤器每天会为每个证券生成一个`True`或`False`值。

## Factor/Classifier Methods

因子和分类器类的各种方法返回过滤器。再次，因为我们还没有看过分类器，现在让我们继续使用因子方法（稍后我们将看看分类器方法）。

Factor.top(n)方法会生成一个过滤器，该过滤器每天对给定因子的前n个证券返回True。以下示例会生成一个筛选器，该筛选器每天返回前200个证券，表明这些证券在所有已知证券的最后收盘价中，排位于前200。

+ 有关返回过滤器的因子方法的完整列表，请参阅此[链接](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor)。

+ 有关返回过滤器的分类器方法的完整列表，请参阅此[链接](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier)。

## Dollar Volume Filter

让我们创建一个过滤器，如果证券的30天平均美元交易量高于10,000,000美元，则返回True。为此，我们首先需要创建`AverageDollarVolume`因子来计算30天的平均美元交易量。

要导入内置的AverageDollarVolume过滤器，我们可以添加到我们用于导入`SimpleMovingAverage`的行。

In [1]:
from zipline.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from zipline.pipeline import Pipeline

然后我们可以实例化因子。

In [2]:
from zipline.pipeline.data import USEquityPricing

In [3]:
dollar_volume = AverageDollarVolume(window_length=30, inputs=[USEquityPricing.close, USEquityPricing.volume])

默认情况下，`AverageDollarVolume`使用`USEquityPricing.close`和`USEquityPricing.volume`作为输入，所以我们不指定它们。(改编后，必须指定输入，否则不会得到正确结果。)

现在我们有一个美元成交量因子，我们可以用布尔表达式创建一个过滤器。以下行创建一个过滤器，其返回True的值为dollar_volume大于10,000,000的证券：

In [4]:
# 更改为10亿
high_dollar_volume = (dollar_volume > 1000000000)

要查看此过滤器的外观，我们可以将其作为列添加到我们的管道中。

In [5]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(
        window_length=30,
        inputs=[USEquityPricing.close, USEquityPricing.volume])

    high_dollar_volume = (dollar_volume > 10000000)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume
        }, )

如果我们创建并运行我们的管道，现在我们有一个列high_dollar_volume，其布尔值对应于每个安全性的表达式结果。

In [6]:
from zipline.research import run_pipeline

In [7]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
result.head(30)

high_dollar_volume  percent_difference
2018-04-10 00:00:00+00:00 平安银行(000001)                True       -6.097735e-02
                          万 科Ａ(000002)                True       -1.204231e-02
                          国农科技(000004)                True        3.964590e-02
                          世纪星源(000005)                True       -2.526891e-02
                          深振业Ａ(000006)                True       -1.376356e-01
                          全新好(000007)                 True       -6.410877e-02
                          神州高铁(000008)                True       -5.493350e-02
                          中国宝安(000009)                True       -8.060563e-03
                          美丽生态(000010)                True       -3.480920e-02
                          深物业A(000011)                True       -1.436390e-02
                          南 玻Ａ(000012)                True       -4.838038e-02
                          沙河股份(000014)                True        3.474035e-02
                          深康佳Ａ(000016)                True        5.793958e-02
                          深中华A(000017)                True       -1.626158e-02
                          神州长城(000018)                True        2.315257e-02
                          深深宝Ａ(000019)               False        6.671771e-03
                          深华发Ａ(000020)                True        2.319089e-03
                          深科技(000021)                 True       -1.909143e-02
                          深赤湾Ａ(000022)               False        2.943425e-16
                          深天地Ａ(000023)                True       -1.019080e-02
                          特 力Ａ(000025)                True       -3.267183e-02
                          飞亚达Ａ(000026)                True        3.204985e-02
                          深圳能源(000027)                True       -4.132943e-03
                          国药一致(000028)                True        1.364167e-02
                          深深房Ａ(000029)               False        1.590293e-16
                          富奥股份(000030)                True       -4.497988e-02
                          中粮地产(000031)               False        0.000000e+00
                          深桑达Ａ(000032)                True        8.790054e-02
                          神州数码(000034)               False       -6.579099e-16
                          中国天楹(000035)               False        1.321694e-16

## Applying a Screen

默认情况下，管道每天为Quantopian数据库中的每个资产生成计算值。 然而，很多时候，我们只关心符合特定标准的证券子集（例如，我们可能只关心有足够的每日交易量来快速填补我们订单的证券）。我们可以通过将筛选器通过`screen`关键字传递给我们的管道来告诉我们的管道忽略过滤器产生`False`的证券。

为了筛选我们的30天平均美元数量大于10,000,000美元证券的管道输出，我们可以简单地将我们的high_dollar_volume过滤器作为屏幕参数。 这就是我们的make_pipeline函数现在的样子：

In [8]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(
        inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(
        window_length=30,
        inputs=[USEquityPricing.close, USEquityPricing.volume])
    high_dollar_volume = (dollar_volume > 100000000)

    return Pipeline(
        columns={'percent_difference': percent_difference},
        screen=high_dollar_volume)

运行此操作将仅为在给定日期通过high_dollar_volume的证券生成输出。 例如，2018年4月10日运行这条管道可以产生约1109个证券。

In [9]:
result = run_pipeline(make_pipeline(), '2018-04-10', '2018-04-10')
print('Number of securities that passed the filter: %d' % len(result))

Number of securities that passed the filter: 1109


## Inverting a Filter

`~`运算符用于反转过滤器，将所有`True`值与`False`交换，反之亦然。例如，我们可以编写以下内容来筛选低美元数量的证券：

low_dollar_volume = ~high_dollar_volume

对于在过去30天内平均美元交易量低于或等于10,000,000美元的所有证券，这将返回True。

在下一课中，我们将介绍如何组合滤镜。